# Exercsie 2.1

In [57]:
import pandas as pd
import plotly.express as px
df = pd.read_csv("data/share-of-individuals-using-the-internet.csv")
df.head()

,Entity,Code,Year,Individuals using the Internet (% of population)
0,Afghanistan,AFG,1990,0.0
1,Afghanistan,AFG,1991,0.0
2,Afghanistan,AFG,1992,0.0
3,Afghanistan,AFG,1993,0.0
4,Afghanistan,AFG,1994,0.0


In [59]:
df_country_cont= px.data.gapminder()[["country", "continent"]]
df_country_cont

,country,continent
0,Afghanistan,Asia
1,Afghanistan,Asia
2,Afghanistan,Asia
3,Afghanistan,Asia
4,Afghanistan,Asia
...,...,...
1699,Zimbabwe,Africa
1700,Zimbabwe,Africa
1701,Zimbabwe,Africa
1702,Zimbabwe,Africa


In [65]:
import duckdb
df_cont = duckdb.query("""--sql
        SELECT d.entity as country, d.code, dc.continent, d.year, "Individuals using the Internet (% of population)" as share_of_pop
        FROM df as d
        LEFT JOIN df_country_cont as dc
             ON d.entity = dc.country
    """).df()
df_cont.head()

,country,Code,continent,Year,share_of_pop
0,Afghanistan,AFG,Asia,1990,0.0
1,Afghanistan,AFG,Asia,1991,0.0
2,Afghanistan,AFG,Asia,1992,0.0
3,Afghanistan,AFG,Asia,1993,0.0
4,Afghanistan,AFG,Asia,1994,0.0


In [66]:
#remove na
df_filtered_cont = df_cont[~pd.isna(df_cont['continent'])]
df_filtered_cont


,country,Code,continent,Year,share_of_pop
0,Afghanistan,AFG,Asia,1990,0.0000
1,Afghanistan,AFG,Asia,1991,0.0000
2,Afghanistan,AFG,Asia,1992,0.0000
3,Afghanistan,AFG,Asia,1993,0.0000
4,Afghanistan,AFG,Asia,1994,0.0000
...,...,...,...,...,...
54145,Sudan,SDN,Africa,2018,24.6000
54146,Sudan,SDN,Africa,2019,25.9000
54147,Sudan,SDN,Africa,2020,26.4383
54148,Sudan,SDN,Africa,2021,27.6336


In [67]:
df_filtered_cont.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51684 entries, 0 to 54149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country       51684 non-null  object 
 1   Code          51684 non-null  object 
 2   continent     51684 non-null  object 
 3   Year          51684 non-null  int64  
 4   share_of_pop  51684 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 2.4+ MB


In [69]:
year_grouped_df = duckdb.query("""--sql
             SELECT continent, year, mean("share_of_pop") as subs_as_shares
             FROM df_filtered_cont
             GROUP BY continent, year
             ORDER BY year, continent
             """).df()
year_grouped_df

,continent,Year,subs_as_shares
0,Africa,1960,0.000000
1,Asia,1960,0.000000
2,Africa,1965,0.000000
3,Asia,1965,0.000000
4,Africa,1970,0.000000
...,...,...,...
200,Oceania,2022,95.303050
201,Africa,2023,18.000000
202,Americas,2023,83.514014
203,Asia,2023,85.980636


In [75]:
df_post_90 = year_grouped_df.query("Year > 1989")
df_post_90

,continent,Year,subs_as_shares
36,Africa,1990,0.000000
37,Americas,1990,0.045829
38,Asia,1990,0.004854
39,Europe,1990,0.121068
40,Oceania,1990,0.292547
...,...,...,...
200,Oceania,2022,95.303050
201,Africa,2023,18.000000
202,Americas,2023,83.514014
203,Asia,2023,85.980636


In [90]:
fig = px.line(df_post_90, x="Year", y="subs_as_shares", color="continent", 
              title= "Share of the population using the internet",
              subtitle="Share of the population who used the internet in the last three months"
              )

fig.update_traces(marker=dict(symbol="circle", size=10))

fig.update_layout(plot_bgcolor="white", width=700,)


fig.show()